# Deskriptive Analyse der Daten des Herzoglich=Sachsen=Gotha= und Altenburgischen Hof= und Adreß=Calenders 

Weiter zu explorativen Analyse -> 

## Anleitung

Das Kapitel der Potentialanalyse der Masterarbeit «Vom Hof= und Adreßcalender zum Datensatz» wird durch mehrere Data Science Notebooks – wie das vorliegende – ergänzt. Diese Notebooks visualisieren Teile der erfassten Datensätze und reichert sie durch zusätzliche Informationen an. Da mit größeren Datenmengen und immer mit den tatsächlichen Daten gearbeitet, können die Abfragen und Visualisierungen mitunter ein wenig Zeit in Anspruch nehmen. Navigiert werden kann über das Inhaltsverzeichnis ☰ in der linken oberen Ecke. Der gesamte Code der Skripte  findet sich hier: s-mamitz/MA-Mast. 

> Technischer Hinweis: In der Regel werden zunächst zwischengespeicherte Analyseergebnisse angezeigt. Um Graphen mit den aktuellsten Daten zu erhalten, müssen die Skripte über den Button oben rechts mit «▸ Run» ausgeführt werden. Unter Umständen muss dies wiederholt werden, wenn die Fehlermeldung erscheint, dass die Maschine noch nicht gestartet ist. Im Folgenden wird Abfrage für Abfrage ausgeführt und visualisiert. Die Visualisierungen können teilweise durch eigene Eingaben manipuliert werden. 

> Hinweis: Die hier gezeigten Visualisierungen sind nur Werkzeuge und kein authentisches Abbild der (historischen) Wirklichkeit. Auch sie sind einer (digitalen) Quellenkritik zu unterziehen. Die vorliegenden Daten wurden in diesem Sinne mehrfach modelliert: Zunächst zeitgenössisch als Datensätze des Amtskalenders, dann als Datentripel für eine Graphdatenbank (FactGrid) und schließlich aggregiert als Visualisierung. Daten aus anderen Projekten werden hinzugezogen; ihre Integrität ist zu prüfen.

## Einführung

### Überblick

Die deskriptive Analyse, also eine Analyse der vorliegenden Daten ohne vertiefende Forschungsfrage, wird durch einen Überblick angeführt. Dieser zeigt, wie viele Daten für den Sachsen-Gotha-Altenburgischen Amtskalender vorliegen, wie diese sich aufteilen und welche der Daten im Projekt der Masterarbeit entstanden. 

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML

# SPARQL Endpoint
endpoint_url = "https://database.factgrid.de/sparql"
sparql = SPARQLWrapper(endpoint_url)

# Google Font "Inter" in Jupyter Notebook einbinden
def setup_fonts():
    display(HTML("""
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:ital,opsz,wght@0,14..32,100..900;1,14..32,100..900&display=swap" rel="stylesheet">
    """))

# Funktion für die erste SPARQL-Abfrage
def fetch_person_data():
    query = """
    SELECT DISTINCT ?item ?itemLabel (MIN(?date) AS ?earliestDate) ?gender WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
      ?item wdt:P2 wd:Q7;
            wdt:P124 ?statement.
      ?statement p:P441 ?refStatement.
      ?refStatement ps:P441 wd:Q76826.
      FILTER(?statement != wd:Q76826)
      OPTIONAL { ?statement wdt:P222 ?date. }
      OPTIONAL { ?item wdt:P154 ?gender. }
    }
    GROUP BY ?item ?itemLabel ?gender
    ORDER BY (?item)
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

# Funktion für die zweite SPARQL-Abfrage
def fetch_project_data():
    query = """
    SELECT DISTINCT ?item ?itemLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
      ?item wdt:P124 ?statement.
      ?statement p:P441 ?refStatement.
      ?refStatement ps:P441 wd:Q76826.
      FILTER(?statement != wd:Q76826)
      ?item wdt:P131 wd:Q960849.
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

# Funktion für die dritte SPARQL-Abfrage
def fetch_non_person_items():
    query = """
    SELECT ?item ?itemLabel ?p2Value ?p2ValueLabel WHERE {
      ?item wdt:P131 wd:Q960849.
      OPTIONAL { ?item wdt:P2 ?p2Value. }
      FILTER NOT EXISTS { ?item wdt:P2 wd:Q7. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

# Verarbeitung der Abfrageergebnisse und Ausgabe
def process_and_display():
    person_data = fetch_person_data()
    project_data = fetch_project_data()
    non_person_data = fetch_non_person_items()
    
    # Verarbeitung der Ergebnisse
    item_count = len(person_data)
    earliest_year = None
    latest_year = None
    gender_counts = { 'Q17': 0, 'Q18': 0 }
    no_gender_count = 0
    
    for item in person_data:
        if 'earliestDate' in item:
            year = int(item['earliestDate']['value'][:4])
            if earliest_year is None or year < earliest_year:
                earliest_year = year
            if latest_year is None or year > latest_year:
                latest_year = year
        if 'gender' in item:
            gender = item['gender']['value']
            if gender == "https://database.factgrid.de/entity/Q17":
                gender_counts['Q17'] += 1
            elif gender == "https://database.factgrid.de/entity/Q18":
                gender_counts['Q18'] += 1
        else:
            no_gender_count += 1
    
    project_count = len(project_data)
    unique_items = set()
    p2_value_counts = {}
    
    for item in non_person_data:
        item_id = item['item']['value']
        unique_items.add(item_id)
        if 'p2ValueLabel' in item:
            label = item['p2ValueLabel']['value']
            p2_value_counts[label] = p2_value_counts.get(label, 0) + 1
    
    item_noperson = len(unique_items)
    sorted_p2_values = sorted(p2_value_counts.items(), key=lambda x: x[1], reverse=True)
    top_p2_values = [label for label, _ in sorted_p2_values[:4]]

    # HTML Ausgabe
    div_style = "color: rgb(51, 64, 82); letter-spacing: -0.09px; font-family: 'Inter', sans-serif;"
    h3_style = "line-height: 28px; font-size: 20px; font-weight: 600;"
    p_style = "line-height: 24px; font-size: 0.875rem; font-weight: normal; letter-spacing: -0.09px;"
    
    html_output = f"""
    <div style="{div_style}">
    <h2 style="{h3_style}">Überblick</h2>
        <p style="{p_style}"><strong>Insgesamt finden sich {item_count} einzigartige Personendatensätze.</strong> Dies für den Zeitraum <strong>{earliest_year}</strong> bis <strong>{latest_year}</strong>. <strong>{project_count} Datensätze</strong> stammen aus dem Projekt. Von den {item_count} Personen sind <strong>{gender_counts['Q17']} weiblich</strong> und <strong>{gender_counts['Q18']} männlich</strong>. <strong>{no_gender_count} Einträge</strong> haben kein Geschlecht angegeben.</p>
        <p style="{p_style}"><strong>{item_noperson} Items</strong> sind keine Personendatensätze. Diese sind hauptsächlich charakterisiert als: <strong>{', '.join(top_p2_values)}</strong>.</p>
    </div>
    """
    display(HTML(html_output))

# Initiales Setup
setup_fonts()
process_and_display()


### Einzelpersonendatensätze

Die folgende Abfrage listet alle im Sachsen-Gotha-Altenburgischen Amtskalender aufgeführt Personen. Bisher sind die Daten vornehmlich im Projekt der vorliegenden Masterarbeit entstanden, können in Zukunft aber auch aus weiteren (Fortsetzungs-)Projekten stammen. Die Tabelle kann sortiert und durchsucht werden. Die Einträge können über die erste Spalte direkt in FactGrid aufgerufen werden. Die Q-Nummern (qid) der letzten Spalte können per Knopfdruck kopiert und an späterer Stelle weiter unten für die Knowledge-Graph-Visualisierung verwendet werden. 

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from IPython.display import display, HTML

def fetch_item_descriptions():
    endpoint_url = "https://database.factgrid.de/sparql"
    sparql = SPARQLWrapper(endpoint_url)

    query = """
    SELECT DISTINCT ?item ?itemLabel ?itemDescription WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
      ?item wdt:P124 ?statement.
      ?statement p:P441 ?refStatement.
      ?refStatement ps:P441 wd:Q76826.
      FILTER(?statement != wd:Q76826)
      OPTIONAL {
        ?item schema:description ?itemDescription.
        FILTER (LANG(?itemDescription) = "de")
      }
    }
    ORDER BY ?item
    """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    items = []
    for result in results["results"]["bindings"]:
        item = result.get("item", {}).get("value", "")
        item_label = result.get("itemLabel", {}).get("value", "")
        item_description = result.get("itemDescription", {}).get("value", "")
        qid = item.split('/')[-1] if item else ""
        item_link = f'<a href="{item}" target="_blank">{qid}</a>' if item else qid
        items.append([item_link, item_label, item_description, qid])

    return pd.DataFrame(items, columns=["Link", "Name", "Beschreibung", "QID"])

def generate_html_table(df):
    html_code = """
    <link rel=\"preconnect\" href=\"https://fonts.googleapis.com\">
    <link rel=\"preconnect\" href=\"https://fonts.gstatic.com\" crossorigin>
    <link href=\"https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap\" rel=\"stylesheet\">
    <link rel=\"stylesheet\" type=\"text/css\" href=\"https://cdn.datatables.net/1.13.4/css/jquery.dataTables.min.css\">
    <script type=\"text/javascript\" charset=\"utf8\" src=\"https://code.jquery.com/jquery-3.6.0.min.js\"></script>
    <script type=\"text/javascript\" charset=\"utf8\" src=\"https://cdn.datatables.net/1.13.4/js/jquery.dataTables.min.js\"></script>
    <style>
body {
    font-family: 'Inter', sans-serif;
}

.scrollable-table {
    max-height: 400px;
    overflow-y: auto;
    overflow-x: hidden;
    border: 1px solid #ccc;
    padding: 10px;
    font-family: 'Inter', sans-serif;
}

table {
    width: 100%;
    border-collapse: collapse;
    table-layout: fixed;
}

th, td {
    padding: 8px 12px;
    border: 1px solid #ddd;
    text-align: left;
    overflow-wrap: break-word;
    white-space: normal !important;
}

th {
    background-color: #f4f4f4;
    font-weight: 600;
}

a {
    color: #007bff;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

td {
    max-width: 250px;
}

tr:nth-child(even) {
    background-color: #ffffff;
}

button {
    background-color: #ffffff;
    color: black;
    border: 1px solid #c2cddc;
    padding: 8px 15px;
    cursor: pointer;
    border-radius: 5px;
    font-weight: 600;
}

button:hover {
    background-color: #edf2f7;
    border: 1px solid #8e9db4;
}
</style>
    <script type=\"text/javascript\">
    $(document).ready(function() {
        $('table').DataTable();
        $('button.copy-qid').on('click', function() {
            var qid = $(this).data('qid');
            navigator.clipboard.writeText(qid).then(function() {
                alert(\"QID \" + qid + \" wurde in die Zwischenablage kopiert!\");
            });
        });
    });
    </script>
    """
    
    def add_copy_button(qid):
        return f'<button class="copy-qid" data-qid="{qid}">qid kopieren</button>'
    
    df['Copy'] = df['QID'].apply(add_copy_button)
    df = df.drop(columns=["QID"])
    table_html = df.to_html(classes="scrollable-table", escape=False, index=False)
    display(HTML(html_code + table_html))

def main():
    df_items = fetch_item_descriptions()
    generate_html_table(df_items)

main()


Link,Name,Beschreibung,Copy
Q134,Johann Michael Böck,"* 1743 in Wien, + 18.07.1793 in Mannheim, Barbier, Schauspieler, Illuminat unter dem Ordensnamen Themisius, Mitglied der Gothaer Loge",qid kopieren
Q1000042,Johann Georg Franck,Student der Universität Halle,qid kopieren
Q1000226,Johann Christian Fröhlich,Student der Universität Halle,qid kopieren
Q1017836,August Friedrich Gottfried Jarislow von Prittwitz,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Page",qid kopieren
Q1017838,Otto Carl August Alexander von Seebach,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Page",qid kopieren
Q1017841,Dorothea Maria Carolina Amalie von Wechmar,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Stiftsfräulein",qid kopieren
Q1017843,Johann Joachim Gotthelf Benedict von Kuntsch,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Fähnrich",qid kopieren
Q1017845,Sophie Luise Johanne Christiane von Reitzenstein,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Kapitular/in",qid kopieren
Q1017847,Gottlob Georg Christoph Ernst von Waldenfelß,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Page",qid kopieren
Q1017850,Ernst Ludwig Adam Walrab von Wangenheim,"um 1770 gelistet im Hofkalender Sachsen-Gotha-Altenburg, u. a. als Page",qid kopieren


## Datensätze im Kontext der Amtskalender

Beide nachstehende Darstellung bieten einen Überblick über die Verteilung der Personendatensätze einmal für jeden Jahrgang auf die vier Etats (Ziviletat Gotha, Hofetat, Ziviletat Altenburg, MilitäretatI), zum anderen auf die inhaltlichen Abschnitte der Behördenliste des Kalenders. Beide Darstellungen können über den Regler die Daten für ein bestimmten Jahr des Untersuchungszeitraumes 1768–1779 an. Wenn eine Person mehrere Ämter inne hat, wird sie in allen entsprechenden Etats oder Abteilungen gezählt. 

🄴 Eingabe für den gewünschten Jahrgang

In [3]:
input_year = 1768

> Kommentar: Um nicht mit den Modellen anderer Projekte zu interferieren zeigt die Grafik nicht die ab 1777 gelisteten Person der Pfarrstellen. Darüberhinaus werden Personen mit Mehrfachanstellungen auch für jeden betreffenden Etat aufgezählt. 

> Hinweis: Zum Fertigstellungszeitpunkt des Projektes lagen die Daten nur für den Jahrgang 1768 vor. Nur für dieses Jahr kann der Inhalt angezeigt werden. 

> Technischer Hinweis: Das sehr kleine Diagramm kann gezoomt und gehovert werden. Um die gezoomte Ansicht zu verlassen, doppelklicken. 

In [4]:
import plotly.graph_objects as go
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import Counter
from IPython.display import display, HTML
import os
import pickle

# Lade die Google-Font "Inter"
display(HTML("""
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Inter:ital,opsz,wght@0,14..32,100..900;1,14..32,100..900&display=swap" rel="stylesheet">
"""))

# SPARQL-Endpunkt
ENDPOINT_URL = "https://database.factgrid.de/sparql"

# Cache-Datei
CACHE_FILE = 'cache.pkl'

# In-Memory-Cache laden (mit pickle)
def load_cache():
    """Lädt den Cache aus der Pickle-Datei, falls vorhanden."""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'rb') as f:
            return pickle.load(f)
    return {}  # Wenn keine Cache-Datei vorhanden ist, gib ein leeres Dictionary zurück.

# Cache speichern (mit pickle)
def save_cache():
    """Speichert den Cache in einer Pickle-Datei."""
    with open(CACHE_FILE, 'wb') as f:
        pickle.dump(cache, f)

# In-Memory-Cache laden
cache = load_cache()

def query_sparql(query):
    """Führt eine SPARQL-Abfrage aus und gibt das Ergebnis zurück."""
    sparql = SPARQLWrapper(ENDPOINT_URL)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

def get_labels_for_targets(targets):
    """Fragt die Labels der Targets über SPARQL ab."""
    if not targets:
        return {}
    
    # Erstelle die VALUES-Klausel für die SPARQL-Abfrage mit einzigartigen Targets
    values_clause = " ".join([f"wd:{target}" for target in set(targets)])
    
    # SPARQL-Abfrage, um Labels zu holen
    query5 = f"""
    SELECT ?item ?itemLabel WHERE {{
      VALUES ?item {{ {values_clause} }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de". }}
    }}
    """
    
    results = query_sparql(query5)
    
    return {res["item"]["value"].split('/')[-1]: res["itemLabel"]["value"] for res in results}

def shorten_label(label, max_length=20):
    """Kürzt das Label, falls es länger als max_length ist, und fügt '...' hinzu."""
    if len(label) > max_length:
        return label[:max_length // 2] + '...' + label[-max_length // 2:]
    return label

def get_calendar_for_year(input_year):
    """Findet die Q-Nummer des Kalenders für das eingegebene Jahr."""
    query6 = f"""
    SELECT ?item WHERE {{
      ?item wdt:P441 wd:Q76826.
      ?item wdt:P222 ?datum.
      FILTER (YEAR(?datum) = {input_year})
    }}
    ORDER BY ?item
    """
    results = query_sparql(query6)
    if results:
        return results[0]['item']['value'].split('/')[-1]  # Extrahiere Q-Nummer
    return None

def get_network_data(input_calendar):
    """Führt die zweite SPARQL-Abfrage aus und gibt eine Liste von Targets zurück."""
    query7 = f"""
    SELECT DISTINCT ?item ?target WHERE {{
      ?item wdt:P2 wd:Q7.
      ?item wdt:P124 wd:{input_calendar}.
      ?item (p:P315/ps:P315) ?intermediate .
      ?intermediate  (p:P428/ps:P428)* ?intermediateStep.
      ?intermediateStep wdt:P428 ?target .

      FILTER (?target IN (wd:Q506752, wd:Q506750, wd:Q1193787, wd:Q506836))
    }}
    ORDER BY ?item
    """
    results = query_sparql(query7)
    return [res["target"]["value"].split('/')[-1] for res in results]

def get_data_for_year(input_year):
    """Lädt die Daten aus dem Cache oder führt eine neue Abfrage aus."""
    if input_year in cache:
        print(f"Daten für {input_year} aus Cache geladen.")
        return cache[input_year]
    
    print(f"Abfrage für das Jahr {input_year} wird durchgeführt (Dauer: ca. 30s)")
    input_calendar = get_calendar_for_year(input_year)
    if not input_calendar:
        print(f"Kein Kalender für das Jahr {input_year} gefunden.")
        return None

    targets = get_network_data(input_calendar)
    cache[input_year] = targets  # Speichere die Daten im Cache
    save_cache()  # Cache nach der neuen Abfrage speichern
    return targets

def visualize_barchart(targets, input_year):
    """Erstellt ein interaktives Balkendiagramm der eindeutigen Targets und deren Häufigkeiten."""
    if not targets:
        print(f"Keine Daten für {input_year} vorhanden.")
        return

    target_counts = Counter(targets)
    target_labels = get_labels_for_targets(list(target_counts.keys()))
    
    sorted_labels = [target_labels.get(target[0], f"Q{target[0]}") for target in sorted(target_counts.items(), key=lambda x: x[1], reverse=True)]
    sorted_counts = [target[1] for target in sorted(target_counts.items(), key=lambda x: x[1], reverse=True)]
    
    display_labels = [shorten_label(label) for label in sorted_labels]
    
    fig = go.Figure([go.Bar(
        x=display_labels, 
        y=sorted_counts, 
        marker_color='#F4C000',
        hovertemplate="<b>%{customdata}</b><br>Anzahl Personen: %{y}<extra></extra>",
        customdata=sorted_labels
    )])

    fig.update_layout(
        title=f"Personendatensätze pro Etat im Jahr {input_year}",
        yaxis_title="Anzahl der Personen",
        xaxis_tickangle=45,
        template="simple_white",  
        font=dict(
            family="Inter, sans-serif",
            size=12,
            color="black"
        ),
        autosize=True
    )

    fig.show()

# Hauptlogik – Verwendet die bereits definierte Variable input_year
try:
    if isinstance(input_year, int) and len(str(input_year)) == 4:
        targets = get_data_for_year(input_year)
        visualize_barchart(targets, input_year)
    else:
        print("Bitte stelle sicher, dass input_year eine gültige vierstellige Jahreszahl ist!")
except NameError:
    print("Die Variable 'input_year' ist nicht definiert. Bitte definiere sie extern.")


Daten für 1768 aus Cache geladen.


In [5]:
import re
import plotly.graph_objects as go
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML

def main(input_year, hover_width="300px"):
    # Google Font laden
    def load_google_font():
        display(HTML("""
        <link rel="preconnect" href="https://fonts.googleapis.com">
        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
        <link href="https://fonts.googleapis.com/css2?family=Inter:ital,opsz,wght@0,14..32,100..900;1,14..32,100..900&display=swap" rel="stylesheet">
        """))
    
    # SPARQL-Abfrage ausführen
    def fetch_data(input_year):
        endpoint_url = "https://database.factgrid.de/sparql"
        sparql = SPARQLWrapper(endpoint_url)
        query = f"""
        SELECT DISTINCT ?item ?itemLabel ?Literal ?Seite ?Seitenspanne ?Position (COUNT(?person) AS ?PersonCount) WHERE {{
          ?item wdt:P2 wd:Q515727.
          ?item wdt:P8 ?calendar.
          ?calendar wdt:P441 wd:Q76826.
          ?calendar wdt:P222 ?datum.
          FILTER (YEAR(?datum) = {input_year})
        
          OPTIONAL {{
            ?person wdt:P124 wd:Q76828.
            ?person wdt:P315 ?item. 
          }}
          
          OPTIONAL {{
            ?item p:P8 ?statement.
            ?statement pq:P35 ?Literal.
          }}
          
          OPTIONAL {{
            ?item p:P8 ?statement.
            ?statement pq:P54 ?Seitenspanne.
          }}
          
          OPTIONAL {{
            ?item p:P8 ?statement.
            ?statement pq:P1208 ?Seite.
          }}
          
          OPTIONAL {{
            ?item p:P8 ?statement.
            ?statement pq:P499 ?Position.
          }}
          
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de". }}
        }}
        GROUP BY ?item ?itemLabel ?Literal ?Seite ?Seitenspanne ?Position
        ORDER BY ?Position
        """
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        return results["results"]["bindings"]
    
    # Funktion zur Kürzung der Labels
    def shorten_label(label, max_length=20):
        """Kürzt das Label, falls es länger als max_length ist, und fügt '...' hinzu."""
        if len(label) > max_length:
            return label[:max_length // 2] + '...' + label[-max_length // 2:]
        return label
    
    # Funktion zur Formatierung der Hover-Labels mit Zeilenumbruch
    def format_hover_label(item, item_uri, literal, person_count, page_range):
        return f"{item} <br><br>Literal: {literal}<br>Personenzahl: {person_count}<br>Seite(n): {page_range}"
    
    # Daten aufbereiten
    def process_data(results):
        data = []
        for res in results:
            item = res.get("itemLabel", {}).get("value", "Unknown")
            item_uri = res.get("item", {}).get("value", "")
            literal = res.get("Literal", {}).get("value", "")
            person_count = int(res.get("PersonCount", {}).get("value", 0))
            page_range = res.get("Seitenspanne", {}).get("value", "")
            position = int(res.get("Position", {}).get("value", 0))
            hover_label = format_hover_label(item, item_uri, literal, person_count, page_range)
            short_label = shorten_label(item)
            data.append({"itemLabel": short_label, "PersonCount": person_count, "Position": position, "hoverLabel": hover_label})
        return pd.DataFrame(data)
    
    # Visualisierung mit Plotly
    def plot_data(df, input_year):
        if df.empty:
            display(HTML(f"<p style='font-family: Inter; line-height: 24px; font-size: 0.875rem; font-weight: normal; color: rgb(51, 64, 82);'> <strong>Zum derzeitigen Zeitpunkt liegen keine Inhaltsdaten für den Jahrgang {input_year} vor.</strong></p>"))
            return
        
        df_sorted = df.sort_values(by="Position")
        items = df_sorted["itemLabel"]
        person_counts = df_sorted["PersonCount"]
        hover_labels = df_sorted["hoverLabel"]
        
        fig = go.Figure()
        fig.add_trace(go.Bar(x=items, y=person_counts, name="Person Count", marker_color='#F4C000', hovertext=hover_labels, hoverinfo="text"))
        
        fig.update_layout(title=f"Inhalt des Kalenders {input_year}",
                          xaxis_title="Kolumnentitel",
                          yaxis_title="Anzahl Personen",
                          barmode='overlay',
                          template="simple_white",
                          font=dict(family="Inter"))
        fig.show()
    
    # Hauptlogik
    load_google_font()
    data = fetch_data(input_year)
    df = process_data(data)
    plot_data(df, input_year)

# Beispielaufruf:
main(input_year, hover_width="250px")


Die folgende Darstellung visualsiert im Gegensatz zu den beiden obigen die eindeutige Personenzahl pro Kalenderjahrgang. Auffällig ist dabei in der Gesamtübersicht der starke Anstieg der Personenzahlen nach der Überarbeitung des Amtskalender 1777 durch Ettinger. Für die Jahre 1768–1779 sind alle Personen des Kalenders übernommen (Stand: März 2025). Etwaige früher oder spätere Personendatensätze garantieren nach derzeitgem Stand keine Vollständigkeit. 

> Kommentar: In dieser Darstellung werde alle Personen, die mehrfach im Kalender genannt werden, nur einmal gezählt. Abgebildet ist also die tatsächliche Zahl verzeichneter Personen.

In [6]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import plotly.express as px
from IPython.display import display, HTML

# Lade die Google-Font "Inter"
def load_google_font():
    display(HTML("""
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:ital,opsz,wght@0,14..32,100..900;1,14..32,100..900&display=swap" rel="stylesheet">
    """))

# SPARQL Abfrage ausführen
def fetch_data_from_sparql():
    sparql = SPARQLWrapper("https://database.factgrid.de/sparql")
    sparql.setQuery("""
    SELECT DISTINCT ?item ?itemLabel (GROUP_CONCAT(DISTINCT ?year; separator=", ") AS ?years) WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
      ?item wdt:P124 ?statement.
      # Verweis auf P441
      ?statement p:P441 ?refStatement.
      ?refStatement ps:P441 wd:Q76826.
      FILTER(?statement != wd:Q76826)

      # Verweis auf P222 und Extrahierung des Jahres
      ?statement p:P222 ?dateStatement.
      ?dateStatement ps:P222 ?date.
      BIND(YEAR(?date) AS ?year)
    }
    GROUP BY ?item ?itemLabel
    ORDER BY ?item
    """)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Daten extrahieren und in DataFrame umwandeln
def process_data(results):
    data = []
    for result in results["results"]["bindings"]:
        item = result["item"]["value"]
        itemLabel = result["itemLabel"]["value"]
        years = result["years"]["value"]
        
        # Aufteilen der Jahre und zählen
        for year in years.split(", "):
            data.append({"year": int(year), "item": item, "itemLabel": itemLabel})

    return pd.DataFrame(data)

# DataFrame analysieren und zurückgeben
def count_items_per_year(df):
    return df.groupby("year").size().reset_index(name="item_count")

# Plotly Line-Graph erstellen
def create_line_plot(df_years):
    min_year = df_years['year'].min()
    max_year = df_years['year'].max()

    fig = px.line(df_years, x="year", y="item_count", title=f"Zahl eindeutiger Personen von {min_year}–{max_year}", markers=True)

    # Farb- und Thema-Anpassungen
    fig.update_traces(
        line=dict(color="#F4C000"),
        hovertemplate="<b>Jahr:</b> %{x}<br>Anzahl der Personen: %{y}<extra></extra>" 
    )
    fig.update_layout(
        template="simple_white",
        xaxis_title="Jahre", 
        yaxis_title="Anzahl der Personen",
        font=dict(
            family="Inter, sans-serif",  # Schriftart auf Inter setzen
            size=12,  # Standardschriftgröße
            color="black"  # Schriftfarbe
        ),
    )

    return fig

# Hauptfunktion, die den gesamten Workflow ausführt
def main():
    load_google_font()  # Google-Font laden

    # Daten aus SPARQL holen
    results = fetch_data_from_sparql()

    # Daten verarbeiten
    df = process_data(results)

    # Anzahl der Items pro Jahr zählen
    df_years = count_items_per_year(df)

    # Diagramm erstellen und anzeigen
    fig = create_line_plot(df_years)
    fig.show()

# Skript ausführen
if __name__ == "__main__":
    main()


## Projektexterner Datenbestand 

### Properties – Messung verfügbarer Informationen

Interessant für den Aspekt der Verknüpfbarkeit der Daten aus den Amtskalendern mit bereits vorhandenen Datensätzen ist die folgende Visualisierung. Dargestellt sind entsprechend ihrer Anzahl bzw. Verbreitung für alle im Projekt bearbeiteten und erstellten Personen-Items alle properties, die eine Aussage mit dem item verbinden. Nicht dargestellt sind properties für qualifier und references. Die Abbildung zeigt also, welche weiteren Informationen prinzipiell vorhanden sein können: Informationen, die im vorliegenden Projekt nicht erfasst wurden, im weiteren Foirschungsprozess aber dennoch hinzugezogen werden können. 

> Technischer Hinweis: Mitunter dauert diese Abfrage etwas länger. Alle properties aller items müssen abgefragt und gezählt werden. 

In [7]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import plotly.express as px
from IPython.display import display, HTML

# Lade die Google-Font "Inter"
def load_google_font():
    display(HTML("""
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:ital,opsz,wght@0,14..32,100..900;1,14..32,100..900&display=swap">
    <style>
        body { font-family: 'Inter', sans-serif; }
    </style>
    """))

# SPARQL Abfrage ausführen
def fetch_data_from_sparql(query):
    sparql = SPARQLWrapper("https://database.factgrid.de/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Daten extrahieren und in DataFrame umwandeln
def process_data(results):
    data = []
    for result in results["results"]["bindings"]:
        data.append({
            "propertyRel": result["propertyRel"]["value"],
            "propertyItem": result["propertyItem"]["value"],
            "propertyItemLabel": result["propertyItemLabel"]["value"],
            "distinctItemCount": int(result["distinctItemCount"]["value"]),
        })
    return pd.DataFrame(data)

# Treemap erstellen
def create_treemap(df_properties):
    fig = px.treemap(
        df_properties,
        path=["propertyItemLabel"],
        values="distinctItemCount",
        hover_data=["propertyRel"],
        title="Treemap der Properties nach Anzahl",
    )

    # Anpassung der Labels
    fig.update_traces(
        texttemplate="<b>%{label}</b><br>%{value}",
        hovertemplate="<b>%{label}</b><br><br>Anzahl: %{value}<br><br>URL: %{customdata[0]}",
    )

    # Setze die Schriftart auf "Inter"
    fig.update_layout(font=dict(family="Inter, sans-serif"))

    return fig

# Hauptfunktion, die den gesamten Workflow ausführt
def main():
    load_google_font()  # Google-Font laden

    # SPARQL-Abfrage
    query8 = """
    SELECT DISTINCT ?propertyRel ?propertyItem ?propertyItemLabel (COUNT(DISTINCT ?item) AS ?distinctItemCount) WHERE {
      ?item wdt:P124 ?statement.
      ?statement p:P441 ?refStatement.
      ?refStatement ps:P441 wd:Q76826.
      FILTER(?statement != wd:Q76826)
      ?item ?propertyRel ?standard.
      ?propertyItem wikibase:directClaim ?propertyRel
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
    }
    GROUP BY ?propertyRel ?propertyItem ?propertyItemLabel
    ORDER BY DESC(?distinctItemCount)
    """

    # Daten aus SPARQL holen
    results = fetch_data_from_sparql(query8)

    # Daten verarbeiten
    df_properties = process_data(results)

    # Treemap erstellen und anzeigen
    fig = create_treemap(df_properties)
    fig.show()

# Skript ausführen
if __name__ == "__main__":
    main()


Dabei stammen die zusätzlichen Informationen unter Umständen aus folgenden Projekten. Die Darstellung basiert auf einer Abfrage der property P131 «Forschungsprojekte, die zu diesem Datensatz beitrugen». Noch ist die stringente und konsistente Verwendung dieser property nicht vollends verbreitet, sodass letztlich nur die Bearbeitungshistorie der Einzel-items Aufschluss geben darüber geben kann, wer welche Informationen tatsächlich zum Datensatz beitrug. 

In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML

# Lade die Google-Font "Inter"
def load_google_font():
    display(HTML("""
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap" rel="stylesheet">
    """))

# SPARQL Abfrage ausführen
def fetch_data_from_sparql(query):
    sparql = SPARQLWrapper("https://database.factgrid.de/sparql")
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    return sparql.query().convert()

# Daten aus den SPARQL-Ergebnissen verarbeiten
def process_data(results):
    data = []
    for result in results["results"]["bindings"]:
        # Datum umformatieren (falls vorhanden)
        raw_date = result.get("beginnDatum", {}).get("value", "")
        try:
            # Entfernen des 'T' und 'Z' und dann Umwandlung in das gewünschte Format
            formatted_date = datetime.strptime(raw_date, "%Y-%m-%dT%H:%M:%SZ").strftime("%d.%m.%Y")
        except ValueError:
            formatted_date = ""  # Falls das Datum ungültig ist
        
        # Klickbare Links erstellen
        website_url = result.get("Website", {}).get("value", "")
        clickable_link = f'<a href="{website_url}" target="_blank">{website_url}</a>' if website_url else ""
        
        data.append({
            "Projekttitel": result.get("ProjektLabel", {}).get("value", ""),
            "FactGrid-Projektseite": clickable_link,
            "Beginn": formatted_date,
            "Anzahl betroffene Items": result.get("AnzahlItems", {}).get("value", "0")
        })
    
    return pd.DataFrame(data)

# CSS für die Tabelle und die Schriftart
def display_table_with_styles(df):
    display(HTML("""
    <style>
    body {
        font-family: 'Inter', sans-serif;
    }

    .scrollable-table {
        max-height: 400px;
        overflow-y: auto;
        border: 1px solid #ccc;
        padding: 10px;
    }

    table {
        width: 100%;
        border-collapse: collapse;
        table-layout: fixed;
    }

    th, td {
        padding: 8px 12px;
        border: 1px solid #ddd;
        text-align: left;
        word-wrap: break-word !important;
        white-space: normal !important;
    }

    th {
        background-color: #f4f4f4;
        font-weight: 600;
    }

    a {
        color: #007bff;
        text-decoration: none;
    }

    a:hover {
        text-decoration: underline;
    }

    td {
        max-width: 250px;
    }
    </style>
    <div class="scrollable-table">
    """ + df.to_html(index=False, escape=False) + """
    </div>
    """))

# Hauptfunktion, die den gesamten Ablauf koordiniert
def main():
    # SPARQL-Abfrage definieren
    query4 = """
    SELECT ?ProjektLabel ?Website ?beginnDatum (COUNT (DISTINCT(?item)) AS ?AnzahlItems) WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }

      ?item wdt:P124 ?statement.
      ?statement p:P441 ?refStatement.
      ?refStatement ps:P441 wd:Q76826.
      FILTER(?statement != wd:Q76826)

      ?item wdt:P131 ?Projekt.
      OPTIONAL { ?Projekt wdt:P852 ?Website. }
      OPTIONAL { ?Projekt wdt:P49 ?beginnDatum. }
    }
    GROUP BY ?ProjektLabel ?Website ?beginnDatum
    ORDER BY DESC(?AnzahlItems)
    """
    
    # SPARQL-Daten abfragen
    results = fetch_data_from_sparql(query4)

    # Daten verarbeiten
    df = process_data(results)

    # Tabelle mit Styles anzeigen
    display_table_with_styles(df)

# Skript ausführen
if __name__ == "__main__":
    load_google_font()  # Google-Font laden
    main()  # Hauptfunktion ausführen


Projekttitel,FactGrid-Projektseite,Beginn,Anzahl betroffene Items
"Marc Eric Mitzscherling, Amtskalender Sachsen-Gotha-Altenburg 1768–1779 (2024)",https://database.factgrid.de/wiki/FactGrid:Hof_Sachsen-Gotha-Altenburg,05.08.2024,3215
"Heino Richard, Ausgangsdaten für Thüringische Genealogien (2019–)",,09.01.2019,708
"Wolfgang Woelk, Geschichte der St. Johannis Loge „Ernst zum Kompass“ im Orient zu Gotha (2022–2023)",,01.01.2022,42
"Martin Gollasch, Forschung zur Frühphase der deutschen Studentenverbindungen, 1750-1815 (2018-2025)",,,30
"Hermann Schüttler, Forschung zu den Mitgliedern des Illuminatenordens (1991/2015).",,,15
"David Löblich, Hallesche Bildungslandschaften im 18. Jahrhundert (2023-)",https://database.factgrid.de/wiki/FactGrid_talk:Halle_im_18._Jahrhundert,01.01.2023,13
"Josef Wäges, Der Aufstieg der Freimauerei",,,4
"Hans Bauer/ Tillmann Kinzel, Die Erik-Amburger-Datenbank im FactGrid (2024–)",https://database.factgrid.de/wiki/FactGrid:Erik-Amburger-Datenbank,,4
Germania Sacra im FactGrid (2020–),,01.01.2020,4
"Erik Liebscher, Gotha im ausgehenden 18. und frühn 19. Jahrhundert (2017–2023)",,01.01.2017,2


### Modelling auf Item-Ebene

Wie sich die verschiedenen properties im Einzelfall um ein items gruppieren, bildet nachstehende interaktive Visualisierung ab. Über das Eingabefeld besteht die Möglichkeit die qid eines (Personen-)Datensatzes aus FactGrid einzufügen und anschließend die properties mit ihren statements abzubilden. Hier werden auch properties der qualifier berücksichtigt. 

🄴 Eingabe für die Q-Nummer der anzuzeigenden Person (Ein eindrückliches Beispiel für viele porperties und statements ist der in FactGrid schon sehr früh angelegte Datensatz zu Heinrich August Ottocar Reichard, Q914)

In [9]:
person_qid = 'Q914'

> Kommentar: Um den Netzwerkgraph übersichtlicher zu gestalten wurden die properties ebenfalls als node bzw. Knoten visualisiert, wenngleich diese im eigentlichen Netzwerk als Kante fungieren müssten. 

> Technischer Hinweis: Unter Umständen funktioniert die folgende Grafik in den Browsern Safari und Chrome nicht. Das Rendering findet auf dem lokalen System statt und kann somit je nach Kapazität länger dauern und mehr Ressourcen verbrauchen. Die Netzwerkgraphik kann gezoomt, verschoben – auch die einzelnen Knoten – und angeklickt werden. 

In [14]:
import requests
import pandas as pd
from pyvis.network import Network
from IPython.display import display, HTML
from datetime import datetime

def format_date(value):
    try:
        return datetime.strptime(value, "%Y-%m-%dT%H:%M:%SZ").strftime("%d.%m.%Y")
    except:
        return value

def fetch_sparql_data(person_qid):
    endpoint = "https://database.factgrid.de/sparql"
    query = f'''
    SELECT ?personLabel ?wd ?wdLabel ?ps_ ?ps_Label ?wdpq ?wdpqLabel ?pq_ ?pq_Label {{
      VALUES (?person) {{(wd:{person_qid})}}
      
      ?person ?p ?statement .
      ?statement ?ps ?ps_ .
      
      ?wd wikibase:claim ?p .
      ?wd wikibase:statementProperty ?ps .
      
      OPTIONAL {{
        ?statement ?pq ?pq_ .
        ?wdpq wikibase:qualifier ?pq .
      }}
      
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de" }}
    }} 
    ORDER BY ?wd ?statement ?ps_
    '''
    
    response = requests.get(endpoint, params={"query": query, "format": "json"})
    data = response.json()
    
    rows = []
    person_label = "Unknown"
    for item in data["results"]["bindings"]:
        if "personLabel" in item:
            person_label = item["personLabel"]["value"]
        row = {key: format_date(item[key]["value"]) if key in item else None for key in ["wd", "wdLabel", "ps_", "ps_Label", "wdpq", "wdpqLabel", "pq_", "pq_Label"]}
        rows.append(row)
    
    return pd.DataFrame(rows), person_label

def create_graph(df, person_qid, person_label):
    net = Network(notebook=True, directed=False)

    # Root Node (Red)
    net.add_node(person_qid, label=person_label, color={'background': '#F88EA0', 'border': '#F43F5E'}, borderWidth=3, size=20)
    
    # Store relationships
    property_nodes = {}
    edge_set = set()
    
    for _, row in df.iterrows():
        wd, wdLabel, ps_, ps_Label, wdpq, wdpqLabel, pq_, pq_Label = row
        
        # Property Nodes (Gray, aggregated)
        if wd not in property_nodes:
            net.add_node(wd, label=wdLabel, color={'background': '#edf2f7', 'border': '#8e9db4'}, borderWidth=3, size=15)
            net.add_edge(person_qid, wd)
            property_nodes[wd] = wdLabel
        
        # Statement Nodes (Yellow, unique)
        statement_node = f"{wd}_{ps_}"
        net.add_node(statement_node, label=ps_Label, color={'background': '#F9E7A4', 'border': '#F4C000'}, borderWidth=3, size=10)
        edge_set.add((wd, statement_node))
        
        # Qualifier Property Nodes (Light Gray, unique per statement)
        if wdpq:
            qualifier_node = f"{statement_node}_{wdpq}"
            net.add_node(qualifier_node, label=wdpqLabel, color={'background': '#F8F8F8', 'border': '#8E9DB4'}, borderWidth=3, size=8)
            edge_set.add((statement_node, qualifier_node))
            
            # Qualifier Value Nodes (Light Yellow, unique per qualifier instance)
            if pq_:
                qualifier_value_node = f"{qualifier_node}_{pq_}"
                net.add_node(qualifier_value_node, label=pq_Label, color={'background': '#FBF5DD', 'border': '#F4C000'}, borderWidth=3, size=6)
                edge_set.add((qualifier_node, qualifier_value_node))
    
    for edge in edge_set:
        net.add_edge(*edge)
    
    net.show("knowledge_graph.html")
    display(HTML("knowledge_graph.html"))

def main():
    #person_qid = "Q914"  # Ersetze durch die gewünschte ID
    df, person_label = fetch_sparql_data(person_qid)
    create_graph(df, person_qid, person_label)

if __name__ == "__main__":
    main()


knowledge_graph.html


### Verbindungen untereinander – Grenzfall zwischen Deskription und Exploration

Verbindungen zwischen Personen

Der Vorteil von Vernetzungen der items untereinander, die aus anderen Projekten hervorgingen, lässt sich in dieser Netzwerkgraphik verdeutlichen. An der Grenzen zwischen Deskription und bereits explorativem Interesse zeigt diese für alle Personendatensätze in Verbindung mit dem Sachsen-Gotha-Altenburgischen Amtskalender alle Verbindungen zu anderen items, die als Mensch charakterisiert werden: Im Zentrum der Darstellung sehr verknüpfte Personengruppen; an der Peripherie der Graphik wenig bis kaum verknüpfte Personen. 

Die Kanten des Netzwerkes zeigen, über welche property die items im Einzelfall verknüpft sind. Um die Navigation zu vereinfachen sind die Kanten gleicher property auch in gleicher Farben markiert. Um etwaige Fehler der Markierung abzufangen zeigen die Kanten auch Label und P-Nummer der entsprechenden property. 

> Technischer Hinweis: Mitunter dauert diese Abfrage etwas länger. Ein Netzwerk für über 1000 Personen muss generiert werden. Unter Umständen funktioniert die folgende Grafik in den Browsern Safari und Chrome nicht. Das Rendering findet auf dem lokalen System statt und kann somit je nach Kapazität länger dauern und mehr Ressourcen verbrauchen. Die Netzwerkgraphik kann gezoomt, verschoben – auch die einzelnen Knoten – und angeklickt werden. 

In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pyvis.network import Network
from collections import defaultdict
from IPython.display import display, HTML
import random

def generate_network():
    # Alle Variablen innerhalb der Funktion
    endpoint_url = "https://database.factgrid.de/sparql"
    query = """
        SELECT DISTINCT ?item ?itemLabel ?prop ?propLabel ?person ?personLabel WHERE {
        SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }

        # Bestimme alle relevanten Personen vorab
        {
            SELECT DISTINCT ?person WHERE {
            ?person wdt:P2 wd:Q7.
            }
        }

        # Finde relevante Items mit den geforderten Verknüpfungen
        ?item wdt:P124 ?statement.
        ?statement p:P441 ?refStatement.
        ?refStatement ps:P441 wd:Q76826.

        # Verbindung zwischen Item und Person
        ?item ?p ?person.

        # Property-Label ermitteln
        ?prop wikibase:directClaim ?p.
        }
        ORDER BY ?prop
    """
    
    # SPARQL-Anfrage senden
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    # Netzwerk initialisieren
    net = Network(notebook=True, directed=False)  # Ungerichtetes Netzwerk
    property_colors = {}  # Speichert die Farben für jede Property
    nodes = set()
    edges = set()
    connections = defaultdict(list)  # Für Rootnodes und ihre Verbindungen
    
    # Vordefinierte Farbenliste
    colors = ['#FF3B30', '#FF9500', '#FFCC00', '#28CD41', '#59ADC4', '#55BEF0', '#007AFF', '#AF52DE', '#FF2D55', '#A2845E', '#8E8E93']
    
    # Eine feste Farbe für jede Property anhand des Property-Labels
    def get_property_color(prop_label):
        if prop_label not in property_colors:
            # Wählen einer Farbe aus der vordefinierten Liste, ggf. HueShift anwenden
            color = colors[len(property_colors) % len(colors)]
            property_colors[prop_label] = color
        return property_colors[prop_label]
    
    # HueShift-Funktion
    def apply_hue_shift(hex_color, shift):
        # Umwandlung von HEX in HSL, Hue-Shift anwenden und dann zurück nach HEX
        h, s, l = hex_to_hsl(hex_color)
        h = (h + shift) % 360  # Hue um den Shift-Wert verschieben, dabei den Farbkreis rotieren
        return hsl_to_hex(h, s, l)
    
    def hex_to_hsl(hex_color):
        # HEX -> RGB
        r, g, b = [int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5)]
        
        # RGB -> HSL
        mx = max(r, g, b)
        mn = min(r, g, b)
        h = s = l = (mx + mn) / 2
        
        if mx == mn:
            h = s = 0
        else:
            d = mx - mn
            s = d / (1 - abs(2 * l - 1)) if l > 0.5 else d / (mx + mn)
            if mx == r:
                h = (g - b) / d + (6 if g < b else 0)
            elif mx == g:
                h = (b - r) / d + 2
            elif mx == b:
                h = (r - g) / d + 4
            h /= 6
        
        return (h * 360, s * 100, l * 100)  # Rückgabe als HSL

    def hsl_to_hex(h, s, l):
        # HSL -> RGB -> HEX
        c = (1 - abs(2 * l / 100 - 1)) * s / 100
        x = c * (1 - abs((h / 60) % 2 - 1))
        m = l / 100 - c / 2
        r, g, b = 0, 0, 0
        
        if 0 <= h < 60:
            r, g, b = c, x, 0
        elif 60 <= h < 120:
            r, g, b = x, c, 0
        elif 120 <= h < 180:
            r, g, b = 0, c, x
        elif 180 <= h < 240:
            r, g, b = 0, x, c
        elif 240 <= h < 300:
            r, g, b = x, 0, c
        elif 300 <= h < 360:
            r, g, b = c, 0, x
        
        r = round((r + m) * 255)
        g = round((g + m) * 255)
        b = round((b + m) * 255)
        
        return f"#{r:02x}{g:02x}{b:02x}"
    
    # Ergebnisse verarbeiten
    for result in results["results"]["bindings"]:
        item = result["item"]["value"].split("/")[-1]
        item_label = result["itemLabel"]["value"]
        person = result["person"]["value"].split("/")[-1]
        person_label = result["personLabel"]["value"]
        prop = result["prop"]["value"].split("/")[-1]
        prop_label = result["propLabel"]["value"]
        
        # Knoten hinzufügen
        if item not in nodes:
            # Gelber Hintergrund für Rootnode (item), dunkelgelber Rand
            net.add_node(item, label=item_label, title=item_label, color="#F9E7A4", borderColor="#F4C000", shape="dot")
            nodes.add(item)
        if person not in nodes:
            # Hellgrauer Hintergrund für Person, dunkler grauer Rand
            net.add_node(person, label=person_label, title=person_label, color="#EDF2F7", borderColor="#8E9DB4", shape="dot")
            nodes.add(person)
        
        # Kantenfarbe basierend auf der Property definieren
        edge_color = get_property_color(prop_label)  # Wählen der Farbe basierend auf dem prop_label
        
        # Falls die vordefinierten Farben nicht ausreichen, wende einen HueShift an
        if len(property_colors) >= len(colors):
            edge_color = apply_hue_shift(edge_color, 50)  # 50 Grad Hue-Shift auf die Farbe anwenden
        
        # Extrahiere den Prop-Wert (z.B. "P84" aus der URL)
        prop_value = prop.split("/")[-1]  # z.B. "P84" extrahieren
        
        # Kanten hinzufügen (keine Pfeile, größere Dicke)
        if (item, person, prop) not in edges:
            # Label für die Kante: Property-Label und Prop-Wert
            edge_label = f"{prop_label} ({prop_value})"
            net.add_edge(item, person, label=edge_label, color=edge_color, width=5)
            edges.add((item, person, prop))
        
        # Rootnodes und deren Verbindungen tracken
        connections[item].append(person)

    # HTML-Visualisierung erstellen
    net.show("Q7network.html")
    display(HTML("Q7network.html"))

# Funktion ausführen
generate_network()

Q7network.html


Verbindungen zwischen Personen und Organisationen (im weitesten Sinne)

Es sind nicht nur Verbindungen zwischen Items möglich, die als Person charakterisiert wurden, sondern beispielhaft auch Verbindungen zwischen Person-Organisation (hier im weitesten Sinne: Organisationstyp Q11214). Die Anzahl der möglichen Verbindungen ist hier wesentlich größer als im Netzwerk Person-Person. Daher wurden nicht nur die Kanten gleicher Eigenschaft in der gleichen Farbe eingefärbt, sondern Organisationsknoten je nach Anzahl der auf sie verweisenden Kanten in der Größe variiert. Die genaue Anzahl an properties wird angezeigt, wenn über der Mauszeiger über dem entsprechenden Organisationsknoten verharrt. 

Gerade hier ist auch eine digitale Quellenkritik notwendig: So sind mit dem Ernestinum in Gotha verhältnismäßig viele Personen verbunden, was sicherlich auch der zentralen Stellung dieser Bildungseinrichtung im Herzogtum geschuldet sein mag, in erster Linie aber auf ein entsprechendes Erschließungsprojekt zurückgeführt werden kann. 

> Technischer Hinweis: Abfrage und Rendering der Daten können ein wenig Zeit in Anspruch nehmen. Über 4000 Knoten und noch mehr Kanten müssen abgefragt und visualisiert werden. Unter Umständen funktioniert die folgende Grafik in den Browsern Safari und Chrome nicht. Das Rendering findet auf dem lokalen System statt und kann somit je nach Kapazität länger dauern und mehr Ressourcen verbrauchen. Die Netzwerkgraphik kann gezoomt, verschoben – auch die einzelnen Knoten – und angeklickt werden. 

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pyvis.network import Network
from collections import defaultdict
from IPython.display import display, HTML
import random

def generate_network():
    def fetch_sparql_results():
        endpoint_url = "https://database.factgrid.de/sparql"
        query = """
        SELECT DISTINCT ?item ?itemLabel ?prop ?propLabel ?orga ?orgaLabel WHERE {
          SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
          
          {
            SELECT DISTINCT ?orga WHERE {
              ?orga (wdt:P2)* wd:Q11214.
            } 
          }
          
          ?item wdt:P124 ?statement.
          ?statement p:P441 ?refStatement.
          ?refStatement ps:P441 wd:Q76826.
          
          ?item ?p ?orga.
          
          ?prop wikibase:directClaim ?p.
        }
        ORDER BY ?prop
        """
        
        sparql = SPARQLWrapper(endpoint_url)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()
    
    def initialize_network():
        net = Network(notebook=True, directed=False, height="800px", width="100%")
        net.barnes_hut(gravity=-5000, central_gravity=0.3, spring_length=200, spring_strength=0.05, damping=0.09)
        return net
    
    def get_property_color(prop_label, property_colors, colors):
        if prop_label not in property_colors:
            color = colors[len(property_colors) % len(colors)]
            property_colors[prop_label] = color
        return property_colors[prop_label]
    
    results = fetch_sparql_results()
    net = initialize_network()
    
    property_colors = {}
    orga_connections = defaultdict(lambda: defaultdict(int))
    bundled_edges = defaultdict(lambda: defaultdict(int))
    nodes = set()
    colors = ['#FF3B30', '#FF9500', '#FFCC00', '#28CD41', '#59ADC4', '#55BEF0', '#007AFF', '#AF52DE', '#FF2D55']
    
    for result in results["results"]["bindings"]:
        item = result["item"]["value"].split("/")[-1]
        item_label = result["itemLabel"]["value"]
        orga = result["orga"]["value"].split("/")[-1]
        orga_label = result["orgaLabel"]["value"]
        prop = result["prop"]["value"].split("/")[-1]
        prop_label = result["propLabel"]["value"]
        
        orga_connections[orga][prop_label] += 1
        bundled_edges[(item, orga)][prop_label] += 1
        
        if item not in nodes:
            net.add_node(item, label=item_label, title=item_label, color="#F9E7A4", borderColor="#F4C000", shape="dot", size=15)
            nodes.add(item)
        if orga not in nodes:
            net.add_node(orga, label=orga_label, title=orga_label, color="#EDF2F7", borderColor="#8E9DB4", shape="dot", size=10)
            nodes.add(orga)
    
    for (item, orga), props in bundled_edges.items():
        edge_labels = [f"{p} ({count})" for p, count in props.items()]
        edge_label = " | ".join(edge_labels)
        edge_color = get_property_color(next(iter(props)), property_colors, colors)
        net.add_edge(item, orga, label=edge_label, color=edge_color, width=2 + sum(props.values()) * 0.1, smooth=True)
    
    for orga, props in orga_connections.items():
        total_connections = sum(props.values())
        orga_label = next((result["orgaLabel"]["value"] for result in results["results"]["bindings"] if result["orga"]["value"].split("/")[-1] == orga), orga)
        tooltip = f"{orga_label}\n" + "\n".join([f"{p} ({count} Items)" for p, count in props.items()])
        
        net.get_node(orga)['size'] = 10 + total_connections * 0.5
        net.get_node(orga)['title'] = tooltip
    
    net.show("orgaNetwork.html")
    display(HTML("orgaNetwork.html"))

generate_network()


orgaNetwork.html


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0992f533-5b72-4f32-9475-729b942a6964' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>